In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
from PIL import Image
import numpy as np
import pandas as pd
import glob
import torch
import matplotlib.pyplot as plt # data visualization
import seaborn as sns # statistical data visualization
%matplotlib inline
import os
!pip install Pillow
import seaborn as sn


from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn import ensemble
from sklearn import tree
import xgboost as xgb

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [4]:
%pip install git+https://github.com/mwalmsley/galaxy_mnist.git
from galaxy_mnist import GalaxyMNIST
#from galaxy_mnist import GalaxyMNISTHighrez

  Cloning https://github.com/mwalmsley/galaxy_mnist.git to /tmp/pip-req-build-13tofl_3
  Running command git clone --filter=blob:none --quiet https://github.com/mwalmsley/galaxy_mnist.git /tmp/pip-req-build-13tofl_3
  Resolved https://github.com/mwalmsley/galaxy_mnist.git to commit 375e0257bfd41acbcb7ba2e0f33c8c04d0d5beef
  Preparing metadata (setup.py) ... done
Note: you may need to restart the kernel to use updated packages.


In [5]:
#  Import galaxyMNIST dataset
train_dataset = GalaxyMNIST(
    root='ML_DP/gal_mnist',
    download=True
)

test_dataset = GalaxyMNIST(
    root='ML_DP/gal_mnist',
    download=True,
    train=False
)

NameError: name 'GalaxyMNIST' is not defined

In [ ]:
images_train=train_dataset.data
y_train = train_dataset.targets
images_val=test_dataset.data
y_val = test_dataset.targets
#galaxies=np.array(data_images)

print(images_train[0].shape)
nsamples , n , nx, ny = images_train.shape
d2_images_train = images_train.reshape((nsamples,n*nx*ny))
nsamples , n , nx, ny = images_val.shape
d2_images_val = images_val.reshape((nsamples,n*nx*ny))

#from sklearn.model_selection import train_test_split
#images_train, images_val, y_train, y_val = train_test_split(images, labels, test_size=0.33, random_state=42)



# ***Gini***

In [ ]:
#ATTENZIONE a runnarlo... ci mette tanto

classifier_gini_nomax = tree.DecisionTreeClassifier(criterion='gini', random_state=0)
classifier_gini_nomax.fit(d2_images_train, y_train)

#vediamo il numero di nodi e la max depth attuale dell'albero
n_nodes = classifier_gini_nomax.tree_.node_count
print(n_nodes)
print(classifier_gini_nomax.tree_.max_depth)

In [ ]:
# vogliamo capire qual è il max depth ideale. facciamo un 5-fold cross-validation
values = np.arange(3,27,3) # il 27 viene dal passo prima. 
from sklearn.model_selection import cross_val_score
cv_scs = {}
for i in values:
    clf = tree.DecisionTreeClassifier(max_depth=i,random_state=0)
    cv_scs[i]= cross_val_score(clf, d2_images_train, y_train, cv=5, n_jobs=-1)

In [ ]:
error_score=[]
for i in cv_scs.values():
    error_score.append(1-i)
plt.plot(cv_scs.keys(),error_score)
plt.show()

In [ ]:
values_new = [6,7,8,9,10,11] # proviamo a vedere se tra 6 e 11 ci sono valori migliori
#from sklearn.model_selection import cross_val_score
cv_scs_new = {}
for i in values_new:
    clf = tree.DecisionTreeClassifier(max_depth=i,random_state=0)
    cv_scs_new[i]= cross_val_score(clf, d2_images_train, y_train, cv=5, n_jobs=-1)

In [ ]:
error_score_new=[]
for i in cv_scs_new.values():
    error_score_new.append(1-i)
plt.plot(cv_scs_new.keys(),error_score_new)
plt.show()

In [ ]:
classifier_gini = tree.DecisionTreeClassifier(criterion='gini', max_depth=9, random_state=0) # la max_depth viene dal passo precedente, ed è 9
classifier_gini.fit(d2_images_train, y_train) # fittiamo con le immagini del train

#vediamo il numero di nodi e la max depth attuale dell'albero
n_nodes = classifier_gini.tree_.node_count
print(n_nodes)
print(classifier_gini.tree_.max_depth)

#depth9: test 0.62 train 0.84 cv=5
#depth12: test 0.60 train 0.93 cv=5

In [ ]:
y_pred_gini = classifier_gini.predict(d2_images_val) # ora usiamo l albero per predire le labels delle immagini in test 

print('Model accuracy score with criterion gini index: {0:0.4f}'. format(accuracy_score(y_val, y_pred_gini)))

In [ ]:
# facciamo il check per eventuale overfit

y_pred_train_gini = classifier_gini.predict(d2_images_train)
print('Training-set accuracy score: {0:0.4f}'. format(accuracy_score(y_train, y_pred_train_gini)))

In [ ]:
plt.figure(figsize=(15,15))

from sklearn import tree

#tree.plot_tree(classifier_gini.fit(d2_images_train, y_train))

In [ ]:
#clf_gini = classifier_gini.fit(d2_images_train, y_train)
#tree.export_graphviz(clf_gini, out_file='tree.dot')

#from six import StringIO  
#import pydot 
#dot_data = StringIO() 
#tree.export_graphviz(clf_gini, out_file=dot_data) 
#graph = pydot.graph_from_dot_data(dot_data.getvalue()) 
#graph[0].write_pdf("galaxytree.pdf") 

In [ ]:
cf_matrix = confusion_matrix(y_val, y_pred_gini)
classes = GalaxyMNIST.classes
df_cm = pd.DataFrame(cf_matrix 
                     , index = [i for i in classes], columns = [i for i in classes])
plt.figure(figsize = (10,6))
sn.heatmap(df_cm, annot=True, fmt='.0f')
#print('Confusion matrix\n\n', cf_matrix)

# ***Entropy***

In [ ]:
#ATTENZIONE
# instantiate the DecisionTreeClassifier model with criterion entropy

classifier_entropy_nomax = tree.DecisionTreeClassifier(criterion='entropy', random_state=0) #max_depth=3
# fit the model
classifier_entropy_nomax.fit(d2_images_train, y_train)


In [ ]:
#vediamo il numero di nodi e la max depth attuale dell'albero
n_nodes = classifier_entropy_nomax.tree_.node_count
print(n_nodes)
print(classifier_entropy_nomax.tree_.max_depth)

In [ ]:
# vogliamo capire qual è il max depth ideal. facciamo un 3-fold cross-validation
values = np.arange(2,20,3) # il 20 viene dal passo prima. 
from sklearn.model_selection import cross_val_score
cv_scs = {}
for i in values:
    clf = tree.DecisionTreeClassifier(criterion='entropy', max_depth=i,random_state=0)
    cv_scs[i]= cross_val_score(clf, d2_images_train, y_train, cv=5, n_jobs=-1)

In [ ]:
error_score=[]
for i in cv_scs.values():
    error_score.append(1-i)
plt.plot(cv_scs.keys(),error_score)
plt.show()

In [ ]:
# vediamo se ci sono valori migliori attorno a 8
values_new = [6,7,8,9,10]
from sklearn.model_selection import cross_val_score
cv_scs_new = {}
for i in values_new:
    clf = tree.DecisionTreeClassifier(criterion='entropy', max_depth=i,random_state=0)
    cv_scs_new[i]= cross_val_score(clf, d2_images_train, y_train, cv=5, n_jobs=-1)

In [ ]:
error_score_new=[]
for i in cv_scs_new.values():
    error_score_new.append(1-i)
plt.plot(cv_scs_new.keys(),error_score_new)
plt.show()

In [ ]:
classifier_entropy = tree.DecisionTreeClassifier(criterion='entropy', max_depth=8, random_state=0) #max_depth vista dal passo precedente
# fit the model
classifier_entropy.fit(d2_images_train, y_train)

In [ ]:
#vediamo il numero di nodi e la max depth attuale dell'albero
n_nodes = classifier_entropy.tree_.node_count
print(n_nodes)
print(classifier_entropy.tree_.max_depth)

In [ ]:
y_pred_entropy = classifier_entropy.predict(d2_images_val)

print('Model accuracy score with criterion entropy index: {0:0.4f}'. format(accuracy_score(y_val, y_pred_entropy)))

y_pred_train_entropy = classifier_entropy.predict(d2_images_train)

print('Training-set accuracy score: {0:0.4f}'. format(accuracy_score(y_train, y_pred_train_entropy)))

In [ ]:
cf_matrix = confusion_matrix(y_val, y_pred_entropy)
classes = GalaxyMNIST.classes
df_cm = pd.DataFrame(cf_matrix 
                     , index = [i for i in classes], columns = [i for i in classes])
plt.figure(figsize = (10,6))
sn.heatmap(df_cm, annot=True, fmt= '.0f')

In [ ]:
# di pochisismo ma è meglio l entropy. aspettato, visto che gini è più veloce (l entropy ha i logaritmi, quindi più pesante)
#gini: train 0.62 test 0.84
#entropy: train 0.63 test 0.78

# ***Bagging***

In [ ]:
bagging_model = ensemble.BaggingClassifier(tree.DecisionTreeClassifier(max_depth = 9), n_estimators = 15, max_samples = 0.50, max_features = 0.50, random_state=42) # bootstrap, di default è true
bagging_model.fit(d2_images_train, y_train)

In [ ]:
y_train_predBag = bagging_model.predict(d2_images_train)

print(confusion_matrix(y_train, y_train_predBag))
print(accuracy_score(y_train, y_train_predBag))

In [ ]:
y_val_predBag = bagging_model.predict(d2_images_val)

print(confusion_matrix(y_val, y_val_predBag))
print(accuracy_score(y_val, y_val_predBag))

In [ ]:
cf_matrix = confusion_matrix(y_val, y_val_predBag)
classes = GalaxyMNIST.classes
df_cm = pd.DataFrame(cf_matrix 
                     , index = [i for i in classes], columns = [i for i in classes])
plt.figure(figsize = (10,6))
sn.heatmap(df_cm, annot=True, fmt= '.0f')
#print('Confusion matrix\n\n', cf_matrix)

# ***XGBoosting***

In [ ]:
xgb_model = xgb.XGBClassifier(objective='multi:softmax', tree_method='hist',  device='cuda', max_depth=3, random_state=42)
xgb_model.fit(d2_images_train, y_train)

#max_depth=3 overfitta: train 0.74 test: 0.97

In [ ]:
y_train_pred = xgb_model.predict(d2_images_train)

print(confusion_matrix(y_train, y_train_pred))
print(accuracy_score(y_train, y_train_pred))

In [ ]:
#xgb_model.fit(d2_images_val, y_train, eval_set=[(X_test, y_test)])

y_val_pred = xgb_model.predict(d2_images_val)

print(confusion_matrix(y_val, y_val_pred))
print(accuracy_score(y_val, y_val_pred))

In [ ]:
cf_matrix = confusion_matrix(y_val, y_val_pred)
classes = GalaxyMNIST.classes
df_cm = pd.DataFrame(cf_matrix 
                     , index = [i for i in classes], columns = [i for i in classes])
plt.figure(figsize = (10,6))
sn.heatmap(df_cm, annot=True, fmt= '.0f')
#print('Confusion matrix\n\n', cf_matrix)